In [38]:
import requests
import mysql.connector
import re

In [39]:
db_cnx = mysql.connector.connect(user="docker", password="docker", host="localhost", database="tcc")

# Inserindo partidos

In [40]:
listagem_partidos_url = "https://dadosabertos.camara.leg.br/api/v2/partidos?ordem=ASC&ordenarPor=sigla&itens=100"

In [41]:
partidos = requests.get(listagem_partidos_url).json()["dados"]

In [42]:
cursor = db_cnx.cursor()
for partido in partidos:

    statement = """
    insert into partido (id, nome, situacao, uri_logo, uri_site, sigla)
        values (%(id)s, %(nome)s, %(situacao)s, %(uri_logo)s, %(uri_site)s, %(sigla)s);
    """
    params = {"id": partido["id"], "nome": partido["nome"], "sigla": partido["sigla"]}
    detalhes = requests.get(partido["uri"]).json()["dados"]
    params["uri_logo"] = detalhes["urlLogo"]
    params["uri_site"] = detalhes["urlWebSite"]
    if params["uri_site"] == "" or params["uri_site"] is None:
        params["uri_site"] = detalhes["urlFacebook"]
    params["situacao"] = detalhes["status"]["situacao"]
    cursor.execute(statement, params)

In [43]:
db_cnx.commit()

In [44]:
cursor.close()

True

In [45]:
db_cnx.close()